# Tutorial on how to open, visualize and extract some features from a .mhd Image

This Tutorial will show how to:
    - Open and read a .mhd image
    - Visualize a .mhd image
    - Read a list of candidates from a .csv file
    - Transform from world coordinates to voxel coordinates
    - Extract some features / patches of candidates and visualize them
To be able to run this tutorial some python libraries / modules need to be installed:
    - Simple ITK: a library for handling and processing medical images
    - Numpy: a fundamental package for scientific computing with Python
    - PIL (Python Imaging Library): a library for adding image processing capabilities to your Python interpreter 
    - Matplotlib: a plotting library for the Python programming language

We start importing required modules / libraries  using the import command from python

In [1]:
import SimpleITK as sitk
import numpy as np
import csv
import os
from PIL import Image
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import glob
import pandas
%matplotlib inline

We define now a function to:
    - Open the image 
    - Store it into a numpy array
    - Extract the following info: Pixel Spacing, Origin
This function takes as input the name of the image and returns:
    - The array corresponding to the image (numpyImage)
    - Origin (numpyOrigin)
    - PixelSpacing (numpySpacing)

To be able to open and read the list of candidates, we need to use the csv python module. 
We define now a function to:
    - Open a csv file
    - Read a csv file
    - Save each line of a csv file
This functions takes as input the name of the csv file and returns:
    - A list of each line of the csv

In [2]:
np_data = np.vstack(filter(lambda x: x.shape[0] == 1034, data))

# Swap columns around to make our lives easier

np_f_data = np.hstack((np_data[:,1:2], np_data[:,0:1], np_data[:,2:]))

NameError: name 'data' is not defined

In [170]:
np.save("./sample_data_2", np_f_data) # Save the data

In [172]:
json.dumps(xml_paths)

'["resources/luna16_annotations/187/029.xml", "resources/luna16_annotations/187/014.xml", "resources/luna16_annotations/187/158.xml", "resources/luna16_annotations/187/172.xml", "resources/luna16_annotations/187/115.xml", "resources/luna16_annotations/187/277.xml", "resources/luna16_annotations/187/048.xml", "resources/luna16_annotations/187/128.xml", "resources/luna16_annotations/187/258.xml", "resources/luna16_annotations/187/106.xml", "resources/luna16_annotations/187/104.xml", "resources/luna16_annotations/187/266.xml", "resources/luna16_annotations/187/064.xml", "resources/luna16_annotations/187/242.xml", "resources/luna16_annotations/187/041.xml", "resources/luna16_annotations/187/268.xml", "resources/luna16_annotations/187/091.xml", "resources/luna16_annotations/187/251.xml", "resources/luna16_annotations/187/086.xml", "resources/luna16_annotations/187/285.xml", "resources/luna16_annotations/187/208.xml", "resources/luna16_annotations/187/034.xml", "resources/luna16_annotations/

In [164]:
subset0 = set(xml_paths) # These are the paths to ignore

In [173]:
# THIS IS THE MAIN FUNCTION
data, xml_paths = process_lidc_annotations(ignore=subset0)

0 :  resources/luna16_annotations/157/162.xml
0 :  resources/luna16_annotations/157/163.xml
0 :  resources/luna16_annotations/157/161.xml
0 :  resources/luna16_annotations/157/160.xml
0 :  resources/luna16_annotations/157/158.xml


KeyboardInterrupt: 

In [165]:
INPUT_FOLDER = './subset9/'
datapoints = os.listdir(INPUT_FOLDER)
datapoints = set((filter(lambda x: ".mhd" in x, datapoints)))

def find_mhd_file(patient_id):
    fname = patient_id + '.mhd'
    if fname in datapoints:
        img_path = INPUT_FOLDER + fname
        return img_path
    else:
        return None

In [163]:
def process_lidc_annotations(only_patient=None, agreement_threshold=0, ignore=set()):
    # lines.append(",".join())
    file_no = 0
    pos_count = 0
    neg_count = 0
    all_lines = []
    total = []
    paths = []
    for anno_dir in [d for d in glob.glob("resources/luna16_annotations/*") if os.path.isdir(d)]:
        xml_paths = glob.glob(anno_dir + "/*.xml")
        for xml_path in xml_paths:
            if xml_path in ignore:
                continue
            print(file_no, ": ",  xml_path)
            pos, neg, extended, data = load_lidc_xml(xml_path=xml_path, only_patient=only_patient, agreement_threshold=agreement_threshold)
            if data is not None:
                paths.append(xml_path)
                total.extend(data)
                print(len(total))
            #if pos is not None:
            #    print("FOUND " + xml_path)
            #    paths.append((xml_path, (pos, neg, extended)))
            #    break
    return total, paths

In [115]:
def load_lidc_xml(xml_path, agreement_threshold=0, only_patient=None, save_nodules=False):
    pos_lines = []
    neg_lines = []
    extended_lines = []
    data = []
    with open(xml_path, 'r') as xml_file:
        markup = xml_file.read()
    xml = BeautifulSoup(markup, features="xml")
    if xml.LidcReadMessage is None:
        return None, None, None, None
    patient_id = xml.LidcReadMessage.ResponseHeader.SeriesInstanceUid.text
    if only_patient is not None:
        if only_patient != patient_id:
            return None, None, None, None

    #print(patient_id)
    src_path = find_mhd_file(patient_id)
    if src_path is None:
        return None, None, None, None

    itk_img = sitk.ReadImage(src_path)
    img_array = sitk.GetArrayFromImage(itk_img)
    num_z, height, width = img_array.shape        #heightXwidth constitute the transverse plane
    origin = np.array(itk_img.GetOrigin())      # x,y,z  Origin in world coordinates (mm)
    spacing = np.array(itk_img.GetSpacing())    # spacing of voxels in world coor. (mm)
    rescale = spacing / 1.00
    
    reading_sessions = xml.LidcReadMessage.find_all("readingSession")
    for reading_session in reading_sessions:
        # print("Sesion")
        nodules = reading_session.find_all("unblindedReadNodule")
        for nodule in nodules:
            nodule_id = nodule.noduleID.text
            # print("  ", nodule.noduleID)
            rois = nodule.find_all("roi")
            x_min = y_min = z_min = 999999
            x_max = y_max = z_max = -999999
            if len(rois) < 2:
                continue

            for roi in rois:
                z_pos = float(roi.imageZposition.text)
                z_min = min(z_min, z_pos)
                z_max = max(z_max, z_pos)
                edge_maps = roi.find_all("edgeMap")
                for edge_map in edge_maps:
                    x = int(edge_map.xCoord.text)
                    y = int(edge_map.yCoord.text)
                    x_min = min(x_min, x)
                    y_min = min(y_min, y)
                    x_max = max(x_max, x)
                    y_max = max(y_max, y)
                if x_max == x_min:
                    continue
                if y_max == y_min:
                    continue

            x_diameter = x_max - x_min
            x_center = x_min + x_diameter / 2
            y_diameter = y_max - y_min
            y_center = y_min + y_diameter / 2
            z_diameter = z_max - z_min
            z_center = z_min + z_diameter / 2
            z_center -= origin[2]
            z_center /= spacing[2]
            
            #print("X Y Z")
            #print(x_center, y_center, z_center)

            x_center_perc = round(x_center / img_array.shape[2], 4)
            y_center_perc = round(y_center / img_array.shape[1], 4)
            z_center_perc = round(z_center / img_array.shape[0], 4)
            diameter = max(x_diameter , y_diameter)
            diameter_perc = round(max(x_diameter / img_array.shape[2], y_diameter / img_array.shape[1]), 4)

            if nodule.characteristics is None:
                print("!!!!Nodule:", nodule_id, " has no charecteristics")
                continue
            if nodule.characteristics.malignancy is None:
                print("!!!!Nodule:", nodule_id, " has no malignacy")
                continue

            malignacy = nodule.characteristics.malignancy.text
            sphericiy = nodule.characteristics.sphericity.text
            margin = nodule.characteristics.margin.text
            spiculation = nodule.characteristics.spiculation.text
            texture = nodule.characteristics.texture.text
            calcification = nodule.characteristics.calcification.text
            internal_structure = nodule.characteristics.internalStructure.text
            lobulation = nodule.characteristics.lobulation.text
            subtlety = nodule.characteristics.subtlety.text
            
            #print(img_array.shape)
            #print(int(z_center))
            #print(int(x_center-16))
            img_crop = img_array[int(z_center), int(y_center-16):int(y_center+16), int(x_center-16):int(x_center+16)]
            img_crop = normalizePlanes(img_crop)
            #plt.imshow(img_crop, cmap='gray')
            #plt.show()

            line = [nodule_id, x_center_perc, y_center_perc, z_center_perc, diameter_perc, malignacy]
            extended_line = [patient_id, nodule_id, x_center_perc, y_center_perc, z_center_perc, diameter_perc, malignacy, sphericiy, margin, spiculation, texture, calcification, internal_structure, lobulation, subtlety ]
            data_line = [diameter_perc, malignacy, sphericiy, margin, spiculation, texture, calcification, internal_structure, lobulation, subtlety] 
            
            
            data.append(np.append([float(i) for i in data_line], img_crop))
            
            #pos_lines.append(line)
            #extended_lines.append(extended_line)

        nonNodules = reading_session.find_all("nonNodule")
        for nonNodule in nonNodules:
            z_center = float(nonNodule.imageZposition.text)
            z_center -= origin[2]
            z_center /= spacing[2]
            x_center = int(nonNodule.locus.xCoord.text)
            y_center = int(nonNodule.locus.yCoord.text)
            nodule_id = nonNodule.nonNoduleID.text
            x_center_perc = round(x_center / img_array.shape[2], 4)
            y_center_perc = round(y_center / img_array.shape[1], 4)
            z_center_perc = round(z_center / img_array.shape[0], 4)
            diameter_perc = round(max(6 / img_array.shape[2], 6 / img_array.shape[1]), 4)
            # print("Non nodule!", z_center)
            line = [nodule_id, x_center_perc, y_center_perc, z_center_perc, diameter_perc, 0]
            neg_lines.append(line)

#     if agreement_threshold > 1:
#         filtered_lines = []
#         for pos_line1 in pos_lines:
#             id1 = pos_line1[0]
#             x1 = pos_line1[1]
#             y1 = pos_line1[2]
#             z1 = pos_line1[3]
#             d1 = pos_line1[4]
#             overlaps = 0
#             for pos_line2 in pos_lines:
#                 id2 = pos_line2[0]
#                 if id1 == id2:
#                     continue
#                 x2 = pos_line2[1]
#                 y2 = pos_line2[2]
#                 z2 = pos_line2[3]
#                 d2 = pos_line1[4]
#                 dist = math.sqrt(math.pow(x1 - x2, 2) + math.pow(y1 - y2, 2) + math.pow(z1 - z2, 2))
#                 if dist < d1 or dist < d2:
#                     overlaps += 1
#             if overlaps >= agreement_threshold:
#                 filtered_lines.append(pos_line1)
#             # else:
#             #     print("Too few overlaps")
#         pos_lines = filtered_lines

    #df_annos = pandas.DataFrame(pos_lines, columns=["anno_index", "coord_x", "coord_y", "coord_z", "diameter", "malscore"])
    #df_annos.to_csv("out/" + patient_id + "_annos_pos_lidc.csv", index=False)
    #df_neg_annos = pandas.DataFrame(neg_lines, columns=["anno_index", "coord_x", "coord_y", "coord_z", "diameter", "malscore"])
    #df_neg_annos.to_csv("out/" + patient_id + "_annos_neg_lidc.csv", index=False)

    # return [patient_id, spacing[0], spacing[1], spacing[2]]
    return pos_lines, neg_lines, extended_lines, data


In [61]:
def normalizePlanes(npzarray):
     
    maxHU = 400.
    minHU = -1000.
 
    npzarray = (npzarray - minHU) / (maxHU - minHU)
    npzarray[npzarray>1] = 1.
    npzarray[npzarray<0] = 0.
    return npzarray

# THE FOLLOWING CODE IS NOT USED ANY MORE

In [59]:
def load_itk_image(filename):
    itkimage = sitk.ReadImage(filename)
    numpyImage = sitk.GetArrayFromImage(itkimage)
     
    numpyOrigin = np.array(list(reversed(itkimage.GetOrigin())))
    numpySpacing = np.array(list(reversed(itkimage.GetSpacing())))
     
    return numpyImage, numpyOrigin, numpySpacing

In [56]:
def readCSV(filename):
    lines = []
    with open(filename, "r") as f:
        csvreader = csv.reader(f)
        for line in csvreader:
            lines.append(line)
    return lines

Since the coordinates of the candidates are given in World Coordinates, we now need to transform from world coordinates to voxel coordinates. 
We define now a function to do that. Please note that the transformation below is only valid if there is no rotation component in the transformation matrix. For all CT images in our dataset, there is no rotation component so that this formula can be used. 
This function takes as inputs:
    - The world coordinates
    - The origin
    - The pixel Spacing
This function returns:
    - Voxel coordinates (voxelCoord)

In [73]:
def worldToVoxelCoord(worldCoord, origin, spacing):
     
    stretchedVoxelCoord = np.absolute(worldCoord - origin)
    voxelCoord = stretchedVoxelCoord / spacing
    return voxelCoord

We want to extract now some features from the candidates. We define some normalized planes to extract views from the candidates

Using the function defined in line 5 we can:
    - Extract patch for each candidate in the list
    - Visualize each patch
    - Save each page as image in .tiff format

In [57]:
loaded_img = dict()
cands = readCSV("./data/candidates.csv")

In [75]:
i = 0
all_data = []
for cand in cands[31300:]:
    fname = cand[0] + ".mhd"
    if fname not in datapoints:
        continue
    if i % 1000 == 0:
        print("Done %d" % i)
    
    i += 1

    img_path = './data/' + fname
    if img_path in loaded_img:
        numpyImage, numpyOrigin, numpySpacing = loaded_img[img_path]
    else:
        numpyImage, numpyOrigin, numpySpacing = load_itk_image(img_path)
        loaded_img[img_path] = (numpyImage, numpyOrigin, numpySpacing)
    
    
    worldCoord = np.asarray([float(cand[3]),float(cand[2]),float(cand[1])])
    voxelCoord = worldToVoxelCoord(worldCoord, numpyOrigin, numpySpacing)
    voxelWidth = 100
    
    print(numpyImage.shape)
    print(voxelCoord)
    break

    
    patch = numpyImage[int(voxelCoord[0]),int(voxelCoord[1]-voxelWidth/2):int(voxelCoord[1]+voxelWidth/2),int(voxelCoord[2]-voxelWidth/2):int(voxelCoord[2]+voxelWidth/2)]
    patch = normalizePlanes(patch)

    outputDir = 'patches/'
    all_data.append(np.append([int(cand[4])], patch.flatten()))
    #plt.imshow(patch, cmap='gray')
    #plt.show()
    #Image.fromarray(patch*255).convert('L').save(os.path.join(outputDir, 'patch_' + str(worldCoord[0]) + '_' + str(worldCoord[1]) + '_' + str(worldCoord[2]) + '.tiff'))

Done 0
(124, 512, 512)
[ 54.76101806 187.20844995 361.89670584]


In [141]:
results = np.vstack(all_data)

ValueError: all the input array dimensions except for the concatenation axis must match exactly

In [143]:
all_data[0].shape

(10001,)

In [17]:
f_all_data = np.vstack(filter(lambda x: x.shape[0] == 10001, all_data))

In [18]:
np.save("./sample_data_2", f_all_data)